# **Morfología de Zarigüeyas Australianas**

### Importando librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve, roc_auc_score

### Cargando la data

In [ ]:
# Leyendo la data en un Panda Dataframe
data = pd.read_csv('possum.csv')

In [ ]:
# Revisando las primeras 5 filas
data.head()

In [ ]:
# Revisando las dimensiones de la data: cuántas filas (observaciones) x cuántas columnas (variables)
data.shape

In [ ]:
# Revisando los tipos de variables
data.info()

### <u>**Comentario:**</u>
#### 104 Observaciones de 14 variables, 4 categóricas (nominales) y 10 numéricas (proporciones).
#### Las variables Edad ('age') y Longitud del Pie ('footlgth') contienen valores nulos (*missing value*, NaN) que deberán ser eliminados durante EDA.
#### Se busca predecir la variable "sex" en términos de otras variables biométricas.

## **Explorando la data**

In [ ]:
# Elimando observaciones con valores nulos.
# inplace=True para que se modifique internamente el DataFrame
data.dropna(inplace=True)

In [ ]:
data.shape

### <u>**Comentario:**</u>
#### Después de eliminar las observaaciones con valores faltantes, el dataset queda con 101 observaciones.

In [ ]:
# Estadística descriptiva de la data
data.describe()

In [ ]:
# Distribución de las variables categóricas
data.describe(include='object')

In [ ]:
# Visualizando la distribución de la variable a predecir: "sex"
sns.set_style("darkgrid")
sns.histplot(data['sex'], stat='percent', color='olivedrab', shrink=0.9).set(title="Distribución de Sexo")
plt.show()

In [ ]:
# Determinar balance de la variable a predecir.
data['sex'].value_counts(normalize=True)

<u>**Comentario:**</u> La variable a predecir, "sex", presenta buen balance: 59% m vs. 41% f.

### **Distribuciones de algunas variables selectas, estratificadas por la variable a predecir: 'sex'**

In [ ]:
# Distribución de la variable 'Pop'
sns.set_style("whitegrid")
sns.histplot(data, x='Pop', stat='percent', shrink=0.9, hue='sex', palette='Set1', alpha=0.5)
plt.show()

In [ ]:
# Solo para visualizar las proporciones conjuntas. Esta celda la puede borrar.
data[['Pop', 'sex']].value_counts(sort=False, normalize=True)

In [ ]:
# Mostrando distribución conjunta de todos los pares de variables numéricas.
sns.pairplot(data[['age','hdlngth','skullw','totlngth','taill','footlgth','earconch','eye','chest','belly','sex']], kind='reg', hue='sex', corner=True)

<u>**Comentario:**</u> Las variables "case", "site" y "Pop" no serán usadas como predictores. Todas las demás variables biométricas: ['age','hdlngth','skullw','totlngth','taill','footlgth','earconch','eye','chest','belly'] serán usadas como predictores para la variable "sex".

### Ejemplos de distribuciones de algunos predictores

In [ ]:
# Distribución conjunta de Edad ('age') vs. Ancho del Cráneo ('skullw'), estratificada por Sexo ('sex')
sns.jointplot(data, x='age', y='skullw', hue='sex', palette='Set1', kind='scatter')
plt.show()

In [ ]:
# Quedan estas celdas solo de referencia, en caso de que prefiera usar estos tipos de gráficos.
# Borre estas celdas si no serán usadas en su historia de datos final.

# Distribución de las variables Edad ('age') y Ancho del Cráneo ('skullw'), estratificada por Sexo ('sex')
sns.set_style("darkgrid")
f, axs = plt.subplots(ncols=2, sharey=True)
sns.histplot(data, x='age',    stat='density', kde=True, hue='sex', palette='Set1', alpha=0.5, ax=axs[0])
sns.histplot(data, x='skullw', stat='density', kde=True, hue='sex', palette='Set1', alpha=0.5, ax=axs[1])
plt.show()

In [ ]:
# Quedan estas celdas solo de referencia, en caso de que prefiera usar estos tipos de gráficos.
# Borre estas celdas si no serán usadas en su historia de datos final.

# Distribución conjunta de las variables Edad ('age') y Ancho del Cráneo ('skullw'), estratificada por Sexo ('sex')
# Puede mostrar solo una distribución conjunta para cada par seleccionado para los ejemplos. La celda restante la puede borrar.
sns.set_style("whitegrid")
f, axs = plt.subplots(ncols=2, sharey=True)
sns.histplot(data, x='age', y='skullw', hue='sex', palette='Set1', alpha=1, ax=axs[0])
sns.kdeplot( data, x='age', y='skullw', hue='sex', palette='Set1', alpha=1, ax=axs[1])
plt.show()

In [ ]:
# Quedan estas celdas solo de referencia, en caso de que prefiera usar estos tipos de gráficos.
# Borre estas celdas si no serán usadas en su historia de datos final.

sns.lmplot(data, x='age', y='skullw', hue='sex', palette='Set1')
plt.show()

## **Entrenamiento del Modelo**

### **One-Hot Encoding**

% El contenido debajo queda solo como explicación. Bórrelo para la historia de datos final.

#### *Dummy Variables* (variables ficticias)
```
Etiqueta = [yes, no]
      Etiqueta_yes, Etiqueta_no
yes        1           0
no         0           1

     Etiqueta_yes
yes      1
no       0

Semaforo: Sem = [v, a, r]
  Sem_v Sem_a Sem_r
v    1    0    0
a    0    1    0
r    0    0    1

  Sem_v Sem_a
v    1    0
a    0    1
r    0    0

```




In [ ]:
# Preparación y separación de la data en entrenamiento y prueba
X = data[['age', 'hdlngth','skullw','totlngth','taill', 'footlgth', 'earconch','eye','chest','belly']]
N = X.shape[0]
# Se utiliza one-hot encoding para la variable a predecir.
# La nueva variable a predecir es ahora sex_m = 1 si el sexo de la zarigüeya es macho, 0 si es hembra.
y = pd.get_dummies(data['sex'], drop_first=True).values.reshape(N,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print("Tamaño de Entrenamiento = {}".format(X_train.shape))
print("Tamaño de Test          = {}".format(X_test.shape))

In [ ]:
# Pruebe primero con los valores por defecto. Si recibe errores o warnings de no convergencia,
# cambie los parámetros listados debajo: tol entre 0.01 y 0.0001; max_iter entre 1000 - 1000000.
# Recuerde el random_state para reproducibilidad.
modelo = LogisticRegression(solver='liblinear', tol=0.0001, max_iter=1000000, random_state=42)
modelo.fit(X_train, y_train)

In [ ]:
# Imprimiendo el modelo resultante
print("Coeficientes del modelo = {}\nIntercepto del modelo = {}".format(modelo.coef_, modelo.intercept_))

#<u>***Modelo resultante:***</u>
$$
SexoM =  \frac{\mathrm{1} }{\mathrm{1} + e^{-[+0.06\cdot{age}
+0.16\cdot{hdlngth}
+0.10\cdot{skullw}
-0.24\cdot{totlngth}
-0.03\cdot{taill}
+0.06\cdot{footlgth}
-0.07\cdot{earconch}
+0.45\cdot{eye}
-0.10\cdot{chest}
-0.11\cdot{belly}
-0.09]} }
$$

In [ ]:
# Evaluando el modelo con data de entrenamiento
acc_train = modelo.score(X_train, y_train)
print("Desempeño en Entrenmiento: Accuracy = {:.5f}".format(acc_train))

In [ ]:
# Evaluando el modelo con data de prueba
acc_test = modelo.score(X_test, y_test)
print("Desempeño en Prueba: Accuracy = {:.5f}".format(acc_test))

In [ ]:
# Prediciendo probabilidad en data de prueba
# Esta función devuelve dos columnas: la probabilidad para la clase 0 (f) y la probabilidad para la clase 1 (m)
y_test_predicted = modelo.predict_proba(X_test)
y_test_predicted

In [ ]:
# Prediciendo clase en data de prueba
# Esta función devuelve la clase: 1 si su predict_proba para clase 1 es >=0.5
y_test_predicted = modelo.predict(X_test)
y_test_predicted

In [ ]:
# Matriz de Confusión en prueba
# Debe recibir y_test_predicted con modelo.predict y no con modelo.predict_proba
confusion_matrix(y_test, y_test_predicted)

In [ ]:
# Reporte de Clasificación para Prueba
print(classification_report(y_test, y_test_predicted))

In [ ]:
print("Accuracy = ",accuracy_score(y_test, y_test_predicted))
print("F1-Score = ",f1_score(y_test, y_test_predicted))

In [ ]:
# Usando el modelo para predecir data nueva
# Se han recibido los siguientes datos biométricos de tres zarigüeyas y se quiere estimar si son machos o hembras
prueba = pd.DataFrame(np.array([[2,	 80, 50, 75, 32, 63, 40, 10, 22, 25],
                                [4,  92, 53, 86, 37, 63, 48, 15, 27, 34],
                                [8, 100, 68, 95, 43, 78, 56, 18, 32, 40]]),
                      columns=['age', 'hdlngth','skullw','totlngth','taill', 'footlgth', 'earconch','eye','chest','belly'])
pred_proba = modelo.predict_proba(prueba)[:,1]
pred_clase = modelo.predict(prueba)
prueba['Sexo_M_Probable'] = pred_proba
prueba['Sexo_M_Estimado'] = pred_clase
prueba

# <u>Conclusión y recomendación:</u>
## El modelo mostrado fue entrenado con una separación de 80%/20% de entrenamiento/prueba, obteniendo una métrica de evaluación:
$$ Accuracy = 57,14% $$
$$ F1-Score = 57,14% $$
## Se recomienda <u>no enviar a producción.</u>
## Es posible que el bajo desempeño se deba al tamaño del dataset. Se recomienda usar algoritmos distintos de clasificación y obtener más datos.

## **INFORMACIÓN EXTRA**
No es necesario incluirlo en su entrega.
(Recuerde borrar todas celdas y comentarios que no sean finales a su entrega).

In [ ]:
# Este método ayuda a calcular una aproximación de la importancia de las variables.
# Su precisión depende de los rangos de valores y unidades de las variables: si son el mismo o comparables, la importancia estimada es bastante acertada.
# Si las variables están en rangos muy disparejos o en unidades distintas, este método debe sustituirse por otro método estadístico de entrenamiento con escalamiento y normalización de variables
# Esta información puede ser útil para explorar otras iteraciones de entrenamiento seleccionando solo las variables más importantes.
# A diferencia de regresión lineal o polinomial, más variables no necesariamente mejora el desempeño del modelo.
np.abs(np.std(X_train)*modelo.coef_[0]).sort_values(ascending=True).plot(kind='barh')

In [ ]:
# Este método calcula la curva ROC y el área bajo la curva: AUC
# Esta métrica es robusta y hasta preferible en la mayoría de los casos de clasificación, especialmente ante desbalance de clases.
y_test_proba = modelo.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, y_test_proba)
auc = roc_auc_score(y_test, y_test_proba)

plt.plot(fpr,tpr, 'b', label='AUC = %0.4f' % auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.title('Curva ROC')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.xlabel('Tasa de Falsos Positivos')
plt.show()

